# Introduction to Python for Data Engineering

This notebook demonstrates how to read data from a public API, process it to create useful information, and store the result in an SQLite database.

It assumes you have read [this article](https://www.startdataengineering.com/post/python-for-de/#extract--load-read-and-write-data-to-any-system).

**Acknowledgment**: Parts of this notebook were developed with the assistance of ChatGPT, an AI language model by OpenAI, to support instructional design and coding tasks related to data engineering concepts.

In [61]:
# Install required packages
!pip install requests

In [62]:
# Import libraries
import requests
import sqlite3
from datetime import datetime

In [63]:
# Read data from CoinGecko API
url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
response = requests.get(url)
data = response.json()

data


{'bitcoin': {'usd': 117708}}

We will extract the Bitcoin price in USD and convert it to ZAR using an estimated exchange rate.
This is an example of turning raw data into useful **information**.

In [64]:
# Function to convert USD to ZAR
def convert_usd_to_zar(usd_amount, rate=18.5):
    return round(usd_amount * rate, 2)

In [65]:
# Extract USD price and apply transformation
supported_codes_url = "https://api.coingecko.com/api/v3/simple/supported_vs_currencies"
response_2 = requests.get(supported_codes_url)
supported_currencies = response_2.json()

currency = input("Please enter the currency code you want to convert to: ").lower().strip()

if currency in supported_currencies:
  print(f"Your code {currency} is valid, converting to your desired currency!")
  url_2 = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=" + currency
  response_2 = requests.get(url_2)
  data_2 = response_2.json()

  cur_price = data_2["bitcoin"][currency]
  usd_price = data["bitcoin"]["usd"]
  timestamp = datetime.now().isoformat()

  print(f"Bitcoin price in USD: {usd_price}")
  print(f"Estimated price in {currency}: {cur_price}")
else:
  print(f"Unfortunately {currency} is not a valid or supported currency code.")

  """
    It is still in the form of an estimation as it is not the exact value based off of the usd value at that exact point in time but it will
    always work regardless of what the exchange rate changes to as it relieson the reference echange rates that coin gecko uses which ensures reliability
    rather than using a hard coded value which is not as future proof as relying on the data retrieved and managed by coin gecko that is live.
    By using a second low cost api quarie it eliminates the need for a separate function that carries out a conversion on the USD value.
  """



Please enter the currency code you want to convert to: zar
Your code zar is valid, converting to your desired currency!
Bitcoin price in USD: 117708
Estimated price in zar: 2071260


In [68]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices_new.db")
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    timestamp TEXT,
    usd REAL,
    currency TEXT,
    cur_price REAL
)
''')

# Insert the new row
cursor.execute("INSERT INTO prices VALUES (?, ?, ?, ?)", (timestamp, usd_price, currency, cur_price))
conn.commit()
conn.close()


In [69]:
# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices_new.db")
cursor = conn.cursor()

# Show the last 5 rows
rows = cursor.execute("SELECT * FROM prices ORDER BY timestamp DESC LIMIT 5").fetchall()
for row in rows:
    print(row)

conn.close()


('2025-08-17T20:44:07.798215', 117708.0, 'zar', 2071260.0)
